In [1]:
%load_ext tensorboard

In [1]:
import tensorflow as tf

In [2]:
import datetime as dt

In [4]:
!rm -rf ./logs

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train, x_test = x_train / 255., x_test / 255.

In [6]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [17]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

logs_dir = "logs/fit/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")

In [18]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir, histogram_freq=1)

In [19]:
history = model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test),
                        callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2202 - accuracy: 0.9345 - val_loss: 0.1146 - val_accuracy: 0.9636
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0987 - accuracy: 0.9703 - val_loss: 0.0797 - val_accuracy: 0.9740
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0672 - accuracy: 0.9787 - val_loss: 0.0687 - val_accuracy: 0.9780
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0529 - accuracy: 0.9830 - val_loss: 0.0665 - val_accuracy: 0.9800
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0406 - accuracy: 0.9870 - val_loss: 0.0645 - val_accuracy: 0.9806


In [20]:
history

In [28]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


# The other approach

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

2022-06-10 16:45:00.518059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [20]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [21]:
train_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

In [25]:
iterator  = iter(train_dataset)
x_train, y_train = iterator.next()

In [26]:
flatten = tf.keras.layers.Flatten()
model = create_model()
labels = model(x_train, training=True)

loss = loss_object(y_train, labels)

In [27]:
y_train.shape, labels.shape

(TensorShape([64]), TensorShape([64, 10]))

In [22]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [23]:
current_time = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [24]:
model = create_model()
EPOCHS = 5

for epoch in range(EPOCHS):
    for x_train, y_train in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for x_test, y_test in test_dataset:
        test_step(model, x_test, y_test)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)


    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.19284872710704803, Accuracy: 94.12833404541016, Test Loss: 0.09545158594846725, Test Accuracy: 97.16999816894531
Epoch 2, Loss: 0.08868951350450516, Accuracy: 97.36166381835938, Test Loss: 0.07864759862422943, Test Accuracy: 97.39999389648438
Epoch 3, Loss: 0.06283343583345413, Accuracy: 98.03333282470703, Test Loss: 0.0662841647863388, Test Accuracy: 97.97999572753906
Epoch 4, Loss: 0.048896875232458115, Accuracy: 98.47000122070312, Test Loss: 0.059823788702487946, Test Accuracy: 98.13999938964844
Epoch 5, Loss: 0.03955984488129616, Accuracy: 98.7550048828125, Test Loss: 0.06302280724048615, Test Accuracy: 97.97999572753906
